In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH11-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH11-Retriever


In [3]:
# 샘플 벡터DB 구축
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 블로그 포스트 로드
loader = WebBaseLoader(
    "https://teddylee777.github.io/openai/openai-assistant-tutorial/", encoding="utf-8"
)

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

# 임베딩 정의
openai_embedding = OpenAIEmbeddings()

# 벡터DB 생성
db = FAISS.from_documents(docs, openai_embedding)

# retriever 생성
retriever = db.as_retriever()

# 문서 검색
query = "OpenAI Assistant APi의 Functions 사용법에 대해 알려주세요."
relevant_docs = retriever.invoke(query)

# 검색된 문서의 개수
len(relevant_docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


4

In [5]:
# 1번 문서를 출력
print(relevant_docs[1].page_content)

Assistant + tools(도구)
Assistants API의 핵심 기능 중 하나는 Code Interpreter, Retrieval, 그리고 사용자 정의 함수(OpenAI Functions)와 같은 도구로 우리가 만든 Assistants가 이러한 도구들을 활용할 수 있도록 설정할 수 있습니다.
아래의 튜토리얼은 각각의 도구가 가지는 역할과 설정하는 방법에 대해 자세히 알아보도록 하겠습니다.
도구1: Code Interpreter(코드 인터프리터)
개요


Code Interpreter를 사용하면 어시스턴트 API가 샌드박스가 적용된 실행 환경에서 Python 코드를 작성하고 실행할 수 있습니다.


이 도구는 다양한 데이터와 형식의 파일을 처리하고 데이터와 그래프 이미지가 포함된 파일을 생성할 수 있습니다.


코드 인터프리터를 사용하면 어시스턴트가 코드를 반복적으로 실행하여 까다로운 코드 및 수학 문제를 해결할 수 있습니다.


요금


# 사용방법

In [6]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

# ChatOpenAI 언어 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

multiquery_retriever = MultiQueryRetriever.from_llm(llm=llm, retriever=db.as_retriever())

In [7]:
# 쿼리에 대한 로깅 설정
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [8]:
# 질문을 정의
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."

# 문서 검색
relevant_docs = multiquery_retriever.invoke(question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['OpenAI Assistant API에서 Functions 기능을 사용하는 방법에 대해 설명해 주세요.  ', 'OpenAI Assistant API의 Functions를 활용하는 방법은 무엇인가요?  ', 'OpenAI Assistant API의 Functions 사용에 대한 가이드를 제공해 주실 수 있나요?']


검색된 문서 개수: 5
OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용


# LCEL Chain 활용하는 방법

In [9]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.(5개의 질문을 생성하도록 프롬프트를 작성하였습니다)
prompt = PromptTemplate.from_template(
    """You are an AI language model assistant.
Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database.
By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.
Your response should be a list of values separated by new lines, eg: `foo\nbar\nbaz\n`

#ORIGINAL QUESTION:
{question}

#Answer in Korean:
"""
)

# 언어 모델 인스턴스를 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# LLMChain을 생성
custom_multiquery_chain = (
    {"question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
)

# 질문을 정의
question = "OpenAI Assistant APi의 Functions 사용법에 대해 알려주세요."

# 체인을 실행하여 생성된 다중 쿼리를 확인합니다.
multi_queries = custom_multiquery_chain.invoke(question)
multi_queries

'OpenAI Assistant API의 Functions 기능을 사용하는 방법에 대해 설명해 주세요.\n\nOpenAI Assistant API에서 Functions를 활용하는 방법에 대해 알고 싶습니다.\n\nOpenAI Assistant API의 Functions 사용에 대한 가이드를 제공해 주실 수 있나요?\n\nOpenAI Assistant API의 Functions 기능을 어떻게 활용할 수 있는지 궁금합니다.\n\nOpenAI Assistant API에서 Functions를 사용하는 절차에 대해 알려주세요.'

In [10]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    llm=custom_multiquery_chain, retriever=db.as_retriever()
)

In [11]:
# 결과
relevant_docs = multiquery_retriever.invoke(question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['OpenAI Assistant API의 Functions 사용법에 대해 설명해 주세요.  ', 'OpenAI Assistant API에서 Functions를 어떻게 활용할 수 있는지 알려주세요.  ', 'OpenAI Assistant API의 Functions 기능에 대한 정보를 제공해 주세요.  ', 'OpenAI Assistant API의 Functions 사용법에 대한 가이드를 부탁드립니다.  ', 'OpenAI Assistant API의 Functions에 대해 자세히 알고 싶습니다.']


검색된 문서 개수: 4
OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용
